<a href="https://colab.research.google.com/github/kesanir/ML-AI-TRADING/blob/main/BLACKLITTERMAN_NW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install yfinance --upgrade --no-cache-dir

In [11]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

In [12]:
def get_market_data(tickers, start_date, end_date):
    """Fetch historical price data using yfinance"""
    data = yf.download(tickers, start=start_date, end=end_date)['Close']
    return data

In [13]:
def calculate_returns(prices):
    """Calculate daily returns and annualized metrics"""
    returns = prices.pct_change().dropna()
    mean_returns = returns.mean() * 252  # Annualized returns
    cov_matrix = returns.cov() * 252     # Annualized covariance
    return mean_returns, cov_matrix

In [20]:
def black_litterman_returns(mean_returns, cov_matrix, risk_aversion, views, view_confidences):
    """
    Calculate Black-Litterman expected returns
    Parameters:
        mean_returns: Prior expected returns
        cov_matrix: Covariance matrix
        risk_aversion: Risk aversion coefficient
        views: Dictionary of views (asset: expected return)
        view_confidences: Dictionary of confidence levels (0-1)
    """
    n_assets = len(mean_returns)
    P = np.zeros((len(views), n_assets))  # Pick matrix
    Q = np.zeros(len(views))              # View returns
    Omega = np.zeros((len(views), len(views)))  # Uncertainty matrix

    # Market equilibrium returns (implied excess returns)
    w_mkt = np.array([0.40, 0.30, 0.20, 0.10])  # Market portfolio weights (SPY, TLT, EEM, GLD)
    pi = risk_aversion * cov_matrix @ w_mkt

    # Populate views matrices
    asset_idx = {'SPY': 0, 'TLT': 1, 'EEM': 2, 'GLD': 3}
    tickers = list(asset_idx.keys()) # Get the list of tickers
    for i, (asset, view_ret) in enumerate(views.items()):
        P[i, asset_idx[asset]] = 1
        Q[i] = view_ret
        # Access covariance matrix using column/index labels
        #Omega[i, i] = (1/view_confidences[asset] - 1) * cov_matrix[asset_idx[asset], asset_idx[asset]]
        Omega[i, i] = (1/view_confidences[asset] - 1) * cov_matrix[asset_idx[asset], asset_idx[asset]]

    # Black-Litterman calculations
    tau = 0.025  # Scaling factor for uncertainty
    cov_inv = np.linalg.inv(cov_matrix)
    omega_inv = np.linalg.inv(Omega)

    term1 = np.linalg.inv(tau * cov_inv + P.T @ omega_inv @ P)
    term2 = tau * cov_inv @ pi + P.T @ omega_inv @ Q

    bl_returns = term1 @ term2
    return bl_returns


In [15]:
def optimize_portfolio(bl_returns, cov_matrix, risk_free_rate=0.02):
    """Calculate optimal portfolio weights"""
    excess_returns = bl_returns - risk_free_rate
    cov_inv = np.linalg.inv(cov_matrix)

    # Optimal weights (tangency portfolio)
    weights = cov_inv @ excess_returns
    weights = weights / np.sum(weights)
    return weights

In [21]:
def main():
    # Parameters
    tickers = ['SPY', 'TLT', 'EEM', 'GLD']
    end_date = datetime(2025, 3, 1)
    start_date = end_date - timedelta(days=252*2)  # 2 years of data
    risk_aversion = 2.5

    # Get market data
    prices = get_market_data(tickers, start_date, end_date)

    # Calculate returns and covariance
    mean_returns, cov_matrix = calculate_returns(prices)

    # Define investor views (example)
    views = {
        'SPY': 0.08,  # 8% expected return for SPY
        'TLT': 0.03,  # 3% expected return for TLT
        'EEM': 0.10,  # 10% expected return for EEM
        'GLD': 0.05   # 5% expected return for GLD
    }
    view_confidences = {
        'SPY': 0.9,   # 90% confidence
        'TLT': 0.7,   # 70% confidence
        'EEM': 0.8,   # 80% confidence
        'GLD': 0.6    # 60% confidence
    }

    # Calculate Black-Litterman returns
    bl_returns = black_litterman_returns(
        mean_returns,
        cov_matrix,
        risk_aversion,
        views,
        view_confidences
    )

    # Optimize portfolio
    weights = optimize_portfolio(bl_returns, cov_matrix)

    # Display results
    print("Black-Litterman Expected Returns:")
    for ticker, ret in zip(tickers, bl_returns):
        print(f"{ticker}: {ret:.4%}")

    print("\nOptimal Portfolio Weights:")
    for ticker, weight in zip(tickers, weights):
        print(f"{ticker}: {weight:.4%}")

    # Calculate portfolio stats
    portfolio_return = np.dot(weights, bl_returns)
    portfolio_vol = np.sqrt(weights.T @ cov_matrix @ weights)
    sharpe_ratio = (portfolio_return - 0.02) / portfolio_vol

    print("\nPortfolio Statistics:")
    print(f"Expected Return: {portfolio_return:.4%}")
    print(f"Volatility: {portfolio_vol:.4%}")
    print(f"Sharpe Ratio: {sharpe_ratio:.4f}")



In [22]:
if __name__ == "__main__":
    main()

[*********************100%***********************]  4 of 4 completed


KeyError: (0, 0)